In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [85]:
data=pd.read_csv("tmdb_5000_movies.csv")
data.head()
data.shape
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [86]:
########데이터 전처리##########
#추천시스템 동작 : 기존에 이미 시청한 영화의 제목을 입력 -> 영화와 비슷한 영화를 추천
#                                                    컨텐츠기반추천
data=data[['id','genres', 'vote_average','vote_count', 'popularity', 
      'title', 'keywords', 'overview']]



"""
필요로 하는 컬럼만 추출, 데이터 표준화/정규화,결측값 처리?
문자데이터(형태소분석, 불용어제거, 대/소문자 통일,...)
feature engineering(특성공학, )

xdata(수집데이터, 입력데이터, 독립변수) -> ydata(출력데이터, 종속변수)
국어, 영어, 수학, 언어능력(50점만점)   수능점수
80     90   100 ,  40   400점
         ...
100명 학생 점수 data를 수집

파생변수 : 기존의 독립변수로부터 연산을 수행하여 만든 변수

x변수들 -> 모델 -> y예측값(y hat)
90 100 95 -> ???예측

타이타닉 data 
승객명단          성별(파생)
Mrs. 이름...        0
Mr. 이름...           1
Dr. ...

"""

'\n필요로 하는 컬럼만 추출, 데이터 표준화/정규화,결측값 처리?\n문자데이터(형태소분석, 불용어제거, 대/소문자 통일,...)\nfeature engineering(특성공학, )\n\nxdata(수집데이터, 입력데이터, 독립변수) -> ydata(출력데이터, 종속변수)\n국어, 영어, 수학, 언어능력(50점만점)   수능점수\n80     90   100 ,  40   400점\n         ...\n100명 학생 점수 data를 수집\n\n파생변수 : 기존의 독립변수로부터 연산을 수행하여 만든 변수\n\nx변수들 -> 모델 -> y예측값(y hat)\n90 100 95 -> ???예측\n\n타이타닉 data \n승객명단          성별(파생)\nMrs. 이름...        0\nMr. 이름...           1\nDr. ...\n\n'

In [87]:
#data['vote_count'].quantile(0.9)#1838
#상위 10% 지점에 해당되는 vote_count 컬럼 값이 1838 이다.
tmp_m=data['vote_count'].quantile(0.89)#1683
data['vote_count']>=tmp_m
tmp_data=data.copy()[data['vote_count']>=tmp_m]
tmp_data.shape

m=data['vote_count'].quantile(0.9)
data=data[data['vote_count']>=m]
#vote_count값에 대해 대략 500등 안에 들어가는 데이터를 추출

#data.info()
#수치데이터(연속형(점수,온도), 범주형(성별,혈액형,학점))
#연속형 수치데이터 -> 구간 -> 범주화

In [88]:
data.shape
data.head()

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [89]:
c=data['vote_average'].mean()
c #6.96
m #1838

1838.4000000000015

In [90]:
#(WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C
def weighted_rating(x, m=m, c=c):
    v=x['vote_count']
    R=x['vote_average']
    return (v/(v+m)*R) + (m/(m+v)*c)
    
data['score']=data.apply(weighted_rating, axis=1)    

In [91]:
data.head()
data[['genres', 'keywords']].head()
#리스트 내부에 딕셔너리 구조로 저장

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":..."


In [92]:
#data['genres'].apply(ast.literal_eval)[0]

In [93]:
#eval()
listStr='[1,2,3,4,5]'
listStr

'[1,2,3,4,5]'

In [94]:
import ast
listData=ast.literal_eval(listStr)
listData

[1, 2, 3, 4, 5]

In [95]:
data['genres']=data['genres'].apply(ast.literal_eval)

In [96]:
data['keywords']=data['keywords'].apply(ast.literal_eval)

In [97]:
data[['genres', 'keywords']]

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."
2,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name..."
3,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,..."
4,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
...,...,...
4291,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...","[{'id': 1366, 'name': 'shotgun'}, {'id': 13000..."
4300,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...","[{'id': 2052, 'name': 'traitor'}, {'id': 6099,..."
4302,"[{'id': 37, 'name': 'Western'}]","[{'id': 801, 'name': 'bounty hunter'}, {'id': ..."
4337,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...","[{'id': 422, 'name': 'vietnam veteran'}, {'id'..."


In [98]:
data.iloc[0]['genres']
# 모든 데이터의 장르 이름을 출력
#     genres
# 0  Action
#    Adventure
#      ...
# 1  Adventure
#     ...

data.iloc[1]['keywords']
#apply함수 활용

[{'id': 270, 'name': 'ocean'},
 {'id': 726, 'name': 'drug abuse'},
 {'id': 911, 'name': 'exotic island'},
 {'id': 1319, 'name': 'east india trading company'},
 {'id': 2038, 'name': "love of one's life"},
 {'id': 2052, 'name': 'traitor'},
 {'id': 2580, 'name': 'shipwreck'},
 {'id': 2660, 'name': 'strong woman'},
 {'id': 3799, 'name': 'ship'},
 {'id': 5740, 'name': 'alliance'},
 {'id': 5941, 'name': 'calypso'},
 {'id': 6155, 'name': 'afterlife'},
 {'id': 6211, 'name': 'fighter'},
 {'id': 12988, 'name': 'pirate'},
 {'id': 157186, 'name': 'swashbuckler'},
 {'id': 179430, 'name': 'aftercreditsstinger'}]

In [103]:
data.iloc[0]['genres']

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [109]:
data['genres']=data['genres'].apply(lambda x: [d['name'] for d in x]).
apply(lambda x: " ".join(x))

In [113]:
data['keywords']=data['keywords'].apply(lambda x: [d['name'] for d in x]).
apply(lambda x: " ".join(x))

In [115]:
data.head(2)
data.to_csv("pre_tmdb_5000_movies.csv", index=False)

In [117]:
#########컨텐츠(성별,장르,토픽,예산,....) 기반 필터링 -> 추천시스템#########
#우리는 장르를 이용해서 추천시스템 만들 예정

In [123]:
#장르(문자열)->숫자로 변환 -> 벡터화 작업
from sklearn.feature_extraction.text import *
#scikit->sk
#scikit-learn.org

In [124]:
data.genres #벡터화

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4291                        Horror Mystery Crime
4300                              Crime Thriller
4302                                     Western
4337                                 Crime Drama
4602                                       Drama
Name: genres, Length: 481, dtype: object

In [180]:
#1. fit_transform함수를 이용한 장르 벡터화
data.head(2)
#장르를 벡터화
count_vector=CountVectorizer(ngram_range=(1,3))
c_vector_genres=count_vector.fit_transform(data['genres'])

In [185]:
print(c_vector_genres.toarray())
print(c_vector_genres.shape)

[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(481, 364)


In [187]:
#2. fit & transform함수를 이용한 장르 벡터화
count_vector=CountVectorizer(ngram_range=(1,3))
count_vector.fit(data['genres'])

CountVectorizer(ngram_range=(1, 3))

In [190]:
len(count_vector.get_feature_names())
count_vector.get_feature_names()
#          <-----------------------364---------------------------->
#          'action'   'action adventure' ... 'western drama adventure'
# 람보         1              1       0      0 ...        1
# ...
# 코만도       1              0       1      0            1
# ...
# 481편

# 모든 각각의 영화들은 364차원 공간에 점(point)으로 표시
# 점의 총 개수? 481개


#          발   손    공부  폭탄 여행
# 람보    1     2     0     3       0
# 코만도 1     2     0     3       0
# 토지    0    1      1     0       2

# 람보를 시청한 관객에게 어떤 영화를 추천?
# cos(람보,코만도)=14 / 14 = 1(똑같다)
# cos(람보,토지) = 2 / 9.xx=0.2xxx
# cos(람보,코만도)  >  cos(람보,토지) 
# => 람보를 본 사람에게 코만도를 추천하자!!!
# ...

['action',
 'action adventure',
 'action adventure animation',
 'action adventure comedy',
 'action adventure crime',
 'action adventure fantasy',
 'action adventure science',
 'action adventure thriller',
 'action adventure war',
 'action adventure western',
 'action comedy',
 'action comedy crime',
 'action comedy drama',
 'action comedy family',
 'action comedy science',
 'action crime',
 'action crime drama',
 'action crime mystery',
 'action crime thriller',
 'action drama',
 'action drama adventure',
 'action drama fantasy',
 'action drama history',
 'action drama horror',
 'action drama mystery',
 'action drama science',
 'action drama thriller',
 'action family',
 'action family fantasy',
 'action family romance',
 'action fantasy',
 'action fantasy comedy',
 'action horror',
 'action mystery',
 'action mystery science',
 'action mystery thriller',
 'action romance',
 'action science',
 'action science fiction',
 'action thriller',
 'action thriller adventure',
 'action thrille

In [207]:
c_vector_genres.toarray().shape
#(481, 364)
#
c_vector_genres.toarray()[3]

#c_vector_genres.toarray()에서 481개 모든 영화들에 대한 쌍을 구성했을때
#가장 코사인유사도가 큰 값을 출력(인덱스도 함께)
#ex) 
print(c_vector_genres.toarray()[0]) #d1
print(c_vector_genres.toarray()[1]) #d2
cos(d1,d2)=d1*d2 / d1요소값 제곱합에 대한 제곱근 * d2요소값 제곱합에 대한 제곱근
...
cos(d1,d481)=d1*d481 / d1요소값 제곱합에 대한 제곱근 * d481요소값 제곱합에 대한 제곱근
=>
최대값에 해당되는 인덱스? 그때의 코사인 유사도?
data[최대값에 해당되는 인덱스][title] 출력 (추천영화)

[1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [206]:
data.iloc[0].genres

'Action Adventure Fantasy Science Fiction'

In [208]:
data.iloc[1].genres

'Adventure Fantasy Action'

Bow란 (Bag of Words, 단어가방): 문서를 구성하는 모든 단어들에 대해 표시하는 방법

문서1:나는 대한민국 사람
문서2:나는 미국 사람
문서3:나는 중국 사람

코퍼스(corpus, 말뭉치) : {나는, 대한민국, 미국, 중국, 사람}
코퍼스 : 단어(도메인) 사전

토지소설 -> 원고 -> 토지 코퍼스:{토지 소설에 등장하는 단어들}
태백산맥 소설 -> 원고 -> 태백산맥 단어들 중에는 토지 코퍼스에 있는 단어도 있지만,
없는 단어도 있을 것이다(예, '지리산' 단어는 토지에는 등장하지 않음 -> OOV라고 함)



Bow: 전체 문서{d1,...,dn}를 구성하는 단어장(vocabulary, {t1,t2,...,tm})를 만든 다음,
각각의 문서(n개)에 대해 단어장에 있는 각각의 단어가 문서 내에 포함되어 있는지,
되어 있지 않은지를 표시하는 방법


In [135]:
from sklearn.feature_extraction import DictVectorizer
#DictVectorizer:단어의 빈도 정보가 담긴 딕셔너리로부터  BOW 벡터를 생성
v=DictVectorizer(sparse=False)
D=[{'A':1, 'B':2},{'B':3,'C':1}]#단어의 빈도 정보가 담긴 딕셔너리
#1번 문서 : {'A':1, 'B':2}, 2번 문서 : {'B':3,'C':1}
x=v.fit_transform(D)
x #bow 출력
#sparse <-> dense
v.feature_names_
v.transform({'C':4, 'D':2})
#{'C':4, 'D':2}딕셔너리를 v변수에 저장된 벡터화 방식으로 변환(bow)해라


array([[0., 0., 4.]])

In [144]:
#CountVectorizer
#문서 -> 단어 토큰 리스트 -> 단어 출현 빈도 -> bow  벡터로 변환
corpus=[ #해당 도메인에 대한 관련 문서들의 집합
    'This is the first document',
    'This is the second second document',
    'And the third one',
    'The last document'
]
cv=CountVectorizer()
cv.fit(corpus)
print(cv.vocabulary_) #0~9번까지 총 10개의 단어로 구성됨
print(cv.transform(corpus).toarray())
#  0 1 2 3 4 5 6 7 8 9
# [0 1 1 1 0 0 0 1 0 1]
# 'This is the first document'

#새로운 문서를 입력받아 bow생성
cv.transform(['Something completely new']).toarray()
cv.transform(['This is the third doc']).toarray()
#3,7,8,9

{'this': 9, 'is': 3, 'the': 7, 'first': 2, 'document': 1, 'second': 6, 'and': 0, 'third': 8, 'one': 5, 'last': 4}
[[0 1 1 1 0 0 0 1 0 1]
 [0 1 0 1 0 0 2 1 0 1]
 [1 0 0 0 0 1 0 1 1 0]
 [0 1 0 0 1 0 0 1 0 0]]


array([[0, 0, 0, 1, 0, 0, 0, 1, 1, 1]], dtype=int64)

In [145]:
corpus=[ #해당 도메인에 대한 관련 문서들의 집합
    'This is the first document',
    'This is the second second document',
    'And the third one',
    'The last document'
]

In [147]:
cv=CountVectorizer(stop_words=['is','the'])
#stop_words:불용어()
cv.fit(corpus)
print(cv.vocabulary_)
print(cv.transform(corpus).toarray())

{'this': 7, 'first': 2, 'document': 1, 'second': 5, 'and': 0, 'third': 6, 'one': 4, 'last': 3}
[[0 1 1 0 0 0 0 1]
 [0 1 0 0 0 2 0 1]
 [1 0 0 0 1 0 1 0]
 [0 1 0 1 0 0 0 0]]


In [151]:
cv=CountVectorizer(stop_words='english')
cv.fit(corpus)
print(cv.vocabulary_)

{'document': 0, 'second': 1}


In [155]:
cv.get_stop_words()

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [156]:
cv=CountVectorizer(analyzer='char')
cv.fit(corpus)
cv.vocabulary_
# corpus=[ #해당 도메인에 대한 관련 문서들의 집합
#     '
# This is the first document',
#     'This is the second second document',
#     'And the third one',
#     'The last document']

{'t': 14,
 'h': 6,
 'i': 7,
 's': 13,
 ' ': 0,
 'e': 4,
 'f': 5,
 'r': 12,
 'd': 3,
 'o': 11,
 'c': 2,
 'u': 15,
 'm': 9,
 'n': 10,
 'a': 1,
 'l': 8}

In [157]:
cv=CountVectorizer(token_pattern="t\w+")
cv.fit(corpus)
cv.vocabulary_

{'this': 2, 'the': 0, 'third': 1}

In [ ]:
#n-그램(Word2Vec):단어의 크기, 
#1(모노)-그램:토큰 하나만 단어, 2(바이)-그램:두개 토큰을 하나의 단어로 사용
#This is the first document 문서를 2(바이)-그램으로 단어를 표현
#This is, is the, the first, first document 


In [158]:
cv=CountVectorizer(ngram_range=(2,2))
cv.fit(corpus)
cv.vocabulary_
# This is the first document',
#     'This is the second second document',
#     'And the third one',
#     'The last document']

{'this is': 11,
 'is the': 2,
 'the first': 6,
 'first document': 1,
 'the second': 8,
 'second second': 5,
 'second document': 4,
 'and the': 0,
 'the third': 9,
 'third one': 10,
 'the last': 7,
 'last document': 3}

In [159]:
cv=CountVectorizer(ngram_range=(1,2))
cv.fit(corpus)
cv.vocabulary_

{'this': 20,
 'is': 5,
 'the': 13,
 'first': 3,
 'document': 2,
 'this is': 21,
 'is the': 6,
 'the first': 14,
 'first document': 4,
 'second': 10,
 'the second': 16,
 'second second': 12,
 'second document': 11,
 'and': 0,
 'third': 18,
 'one': 9,
 'and the': 1,
 'the third': 17,
 'third one': 19,
 'last': 7,
 'the last': 15,
 'last document': 8}

In [161]:
cv=CountVectorizer(ngram_range=(1,2), token_pattern="t\w+")
cv.fit(corpus)
cv.vocabulary_

{'this': 3, 'the': 0, 'this the': 4, 'third': 2, 'the third': 1}

In [175]:
corpus=['This is the first document',
    'This is the second second second second second document',
    'And the third one',
    'The last document']

#빈도수 기반 
cv=CountVectorizer(min_df=2)
cv.fit(corpus)
cv.vocabulary_

{'this': 3, 'is': 1, 'the': 2, 'document': 0}

In [178]:
cv.transform(corpus).toarray()
#cv.transform(corpus).toarray().sum(axis=0) #[3, 2, 4, 2]

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 1, 0],
       [1, 0, 1, 0]], dtype=int64)

In [121]:
#eval함수:파이썬 내장함수
eval("(5*2)/4")
eval("max([1,2,3])")
#eval함수는 보안에 취약 -> 보안에 강한 함수 필요성 -> literal_eval함수 등장
s={'a':3,'b':5}
s['a']


3

In [122]:
s2="{'a':3,'b':5}"
ast.literal_eval(s2)['a']

3

In [83]:
# eval("10*2")
# ast.literal_eval("10*2")
#literal_eval함수는 파이썬 기본 자료형에 대해서만 지원(매우 엄격)
#간단한 수식등은 eval 함수 사용

ValueError: malformed node or string: <_ast.BinOp object at 0x000001824D0FF0D0>